carregar os dados no ambiente de execução

In [ ]:
import pandas as pd

# Specify dtype for column 2 to handle mixed types
df = pd.read_csv('/content/city_temperature.csv', dtype={'column 2': float})
df = df['AvgTemperature'].replace(-99, pd.NA).dropna()
df.to_csv('dados.csv', index=False, header=False)

definir centróides por sorteio com semente fixa


In [ ]:
%%writefile inicializar_centroides.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

static int count_rows(const char *path) {
    FILE *f = fopen(path, "r");
    if (!f) {
        fprintf(stderr, "ERRO: Nao foi possivel abrir o arquivo %s\n", path);
        exit(1);
    }
    int rows = 0;
    char line[1024];
    while (fgets(line, sizeof(line), f)) {
        rows++;
    }
    fclose(f);
    return rows;
}

// Lê os dados de um CSV de uma coluna para um array de doubles.
static double *read_csv_1col(const char *path, int *n_out) {
    int N = count_rows(path);
    if (N <= 0) {
        fprintf(stderr, "ERRO: Arquivo %s esta vazio ou nao pode ser lido.\n", path);
        exit(1);
    }

    double *data = (double *)malloc(N * sizeof(double));
    if (!data) {
        fprintf(stderr, "ERRO: Falha ao alocar memoria para os dados.\n");
        exit(1);
    }

    FILE *f = fopen(path, "r");
    if (!f) {
        fprintf(stderr, "ERRO: Nao foi possivel abrir o arquivo %s\n", path);
        free(data);
        exit(1);
    }

    int i = 0;
    while (i < N && fscanf(f, "%lf", &data[i]) == 1) {
        i++;
    }
    fclose(f);

    *n_out = i; // Retorna o número de pontos efetivamente lidos
    return data;
}

// --- FUNÇÃO PARA ESCOLHER CENTRÓIDES INICIAIS ---
void escolher_centroides_iniciais(const double *dados, int N, int K, double *centroides, unsigned int semente) {
    srand(semente); //semente fixa

    //array para marcar os índices já escolhidos para evitar repetição.
    int *indices_escolhidos = (int *)calloc(N, sizeof(int));
    if (!indices_escolhidos) {
        fprintf(stderr, "ERRO: Falha ao alocar memoria para indices.\n");
        exit(1);
    }

    printf("Escolhendo %d centroides iniciais a partir de %d pontos...\n", K, N);

    for (int i = 0; i < K; i++) {
        int indice_aleatorio;
        do {
            indice_aleatorio = rand() % N; // Gera um índice entre 0 e N-1
        } while (indices_escolhidos[indice_aleatorio] == 1); // Repete se já foi escolhido

        //marca o índice como escolhido e copia o ponto de dado para o array de centróides.
        indices_escolhidos[indice_aleatorio] = 1;
        centroides[i] = dados[indice_aleatorio];
    }

    free(indices_escolhidos);
}

int main(int argc, char **argv) {
    if (argc < 2) {
        printf("Uso: %s <dados.csv>\n", argv[0]);
        return 1;
    }

    const char *arquivo_dados = argv[1];
    const int K = 16; // qtd centroides
    const unsigned int SEMENTE_FIXA = 42;

    // Carrega os dados do arquivo CSV carregado
    int N = 0;
    double *dados = read_csv_1col(arquivo_dados, &N);
    printf("Carregados %d pontos do arquivo %s\n", N, arquivo_dados);

    if (K > N) {
        fprintf(stderr, "ERRO: O numero de centroides (K=%d) nao pode ser maior que o numero de pontos (N=%d).\n", K, N);
        free(dados);
        return 1;
    }

    // Aloca memória para os centróides
    double *centroides = (double *)malloc(K * sizeof(double));
    if (!centroides) {
        fprintf(stderr, "ERRO: Falha ao alocar memoria para os centroides.\n");
        free(dados);
        return 1;
    }

    // Escolhe os centróides iniciais
    escolher_centroides_iniciais(dados, N, K, centroides, SEMENTE_FIXA);

    // Imprime os centróides escolhidos para verificação
    printf("\nCentroides iniciais escolhidos (com semente %u):\n", SEMENTE_FIXA);
    for (int i = 0; i < K; i++) {
        printf("%f\n",centroides[i]);
    }

    // Libera a memória alocada
    free(dados);
    free(centroides);

    return 0;
}

executar o código

In [ ]:
%%shell

gcc inicializar_centroides.c -o inicializar_centroides -lm
./inicializar_centroides dados.csv

escrever em um arquivo csv

In [ ]:
%%writefile centroides_iniciais.csv
50.500000
88.300000
41.500000
36.400000
15.800000
79.200000
86.100000
77.200000
85.200000
84.200000
71.200000
77.300000
83.300000
61.100000
53.200000
36.600000

# Versão Sequencial

kmeans_1d_naive.c
   K-means 1D (C99), implementação "naive":
   - Lê X (N linhas, 1 coluna) e C_init (K linhas, 1 coluna) de CSVs sem cabeçalho.
   - Itera assignment + update até max_iter ou variação relativa do SSE < eps.
   - Salva (opcional) assign (N linhas) e centróides finais (K linhas).

In [ ]:
%%writefile kmeans_1d_naive.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

/* ---------- util CSV 1D: cada linha tem 1 número ---------- */
static int count_rows(const char *path){
    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); exit(1); }
    int rows=0; char line[8192];
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(!only_ws) rows++;
    }
    fclose(f);
    return rows;
}

static double *read_csv_1col(const char *path, int *n_out){
    int R = count_rows(path);
    if(R<=0){ fprintf(stderr,"Arquivo vazio: %s\n", path); exit(1); }
    double *A = (double*)malloc((size_t)R * sizeof(double));
    if(!A){ fprintf(stderr,"Sem memoria para %d linhas\n", R); exit(1); }

    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); free(A); exit(1); }

    char line[8192];
    int r=0;
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(only_ws) continue;

        /* aceita vírgula/ponto-e-vírgula/espaco/tab, pega o primeiro token numérico */
        const char *delim = ",; \t";
        char *tok = strtok(line, delim);
        if(!tok){ fprintf(stderr,"Linha %d sem valor em %s\n", r+1, path); free(A); fclose(f); exit(1); }
        A[r] = atof(tok);
        r++;
        if(r>R) break;
    }
    fclose(f);
    *n_out = R;
    return A;
}

static void write_assign_csv(const char *path, const int *assign, int N){
    if(!path) return;
    FILE *f = fopen(path, "w");
    if(!f){ fprintf(stderr,"Erro ao abrir %s para escrita\n", path); return; }
    for(int i=0;i<N;i++) fprintf(f, "%d\n", assign[i]);
    fclose(f);
}

static void write_centroids_csv(const char *path, const double *C, int K){
    if(!path) return;
    FILE *f = fopen(path, "w");
    if(!f){ fprintf(stderr,"Erro ao abrir %s para escrita\n", path); return; }
    for(int c=0;c<K;c++) fprintf(f, "%.6f\n", C[c]);
    fclose(f);
}

/* ---------- k-means 1D ---------- */
/* assignment: para cada X[i], encontra c com menor (X[i]-C[c])^2 */
static double assignment_step_1d(const double *X, const double *C, int *assign, int N, int K){
    double sse = 0.0;
    for(int i=0;i<N;i++){
        int best = -1;
        double bestd = 1e300;
        for(int c=0;c<K;c++){
            double diff = X[i] - C[c];
            double d = diff*diff;
            if(d < bestd){ bestd = d; best = c; }
        }
        assign[i] = best;
        sse += bestd;
    }
    return sse;
}

/* update: média dos pontos de cada cluster (1D)
   se cluster vazio, copia X[0] (estratégia naive) */
static void update_step_1d(const double *X, double *C, const int *assign, int N, int K){
    double *sum = (double*)calloc((size_t)K, sizeof(double));
    int *cnt = (int*)calloc((size_t)K, sizeof(int));
    if(!sum || !cnt){ fprintf(stderr,"Sem memoria no update\n"); exit(1); }

    for(int i=0;i<N;i++){
        int a = assign[i];
        cnt[a] += 1;
        sum[a] += X[i];
    }
    for(int c=0;c<K;c++){
        if(cnt[c] > 0) C[c] = sum[c] / (double)cnt[c];
        else           C[c] = X[0]; /* simples: cluster vazio recebe o primeiro ponto */
    }
    free(sum); free(cnt);
}

static void kmeans_1d(const double *X, double *C, int *assign,
                      int N, int K, int max_iter, double eps,
                      int *iters_out, double *sse_out)
{
    FILE *sse_file = fopen("sse_evolution_seq.csv", "w");
    double prev_sse = 1e300;
    double sse = 0.0;
    int it;
    for(it=0; it<max_iter; it++){
        sse = assignment_step_1d(X, C, assign, N, K);
        fprintf(sse_file, "%d,%.6f\n", it, sse);
        /* parada por variação relativa do SSE */
        double rel = fabs(sse - prev_sse) / (prev_sse > 0.0 ? prev_sse : 1.0);
        printf("Iteration %d, SSE: %f\n", it + 1, sse); // Added this line back
        if(rel < eps){ it++; break; }
        update_step_1d(X, C, assign, N, K);
        prev_sse = sse;

    }
    *iters_out = it;
    *sse_out = sse;
    fclose(sse_file);
}

/* ---------- main ---------- */
int main(int argc, char **argv){
    if(argc < 3){
        printf("Uso: %s dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [assign.csv] [centroids.csv]\n", argv[0]);
        printf("Obs: arquivos CSV com 1 coluna (1 valor por linha), sem cabeçalho.\n");
        return 1;
    }
    const char *pathX = argv[1];
    const char *pathC = argv[2];
    int max_iter = (argc>3)? atoi(argv[3]) : 50;
    double eps   = (argc>4)? atof(argv[4]) : 1e-4;
    const char *outAssign   = (argc>5)? argv[5] : NULL;
    const char *outCentroid = (argc>6)? argv[6] : NULL;

    if(max_iter <= 0 || eps <= 0.0){
        fprintf(stderr,"Parâmetros inválidos: max_iter>0 e eps>0\n");
        return 1;
    }

    int N=0, K=0;
    double *X = read_csv_1col(pathX, &N);
    double *C = read_csv_1col(pathC, &K);
    int *assign = (int*)malloc((size_t)N * sizeof(int));
    if(!assign){ fprintf(stderr,"Sem memoria para assign\n"); free(X); free(C); return 1; }

    clock_t t0 = clock();
    int iters = 0; double sse = 0.0;
    kmeans_1d(X, C, assign, N, K, max_iter, eps, &iters, &sse);
    clock_t t1 = clock();
    double ms = 1000.0 * (double)(t1 - t0) / (double)CLOCKS_PER_SEC;

    printf("K-means 1D (naive)\n");
    printf("N=%d K=%d max_iter=%d eps=%g\n", N, K, max_iter, eps);
    printf("Iterações: %d | SSE final: %.6f | Tempo: %.1f ms\n", iters, sse, ms);

    write_assign_csv(outAssign, assign, N);
    write_centroids_csv(outCentroid, C, K);

    free(assign); free(X); free(C);
    return 0;
}


executar o código.
- Para compilar: gcc -O2 -std=c99 kmeans_1d_naive.c -o kmeans_1d_naive -lm
- Uso:      ./kmeans_1d_naive dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [assign.csv] [centroids.csv]

In [ ]:
%%shell

gcc -O2 -std=c99 kmeans_1d_naive.c -o kmeans_1d_naive -lm
./kmeans_1d_naive dados.csv centroides_iniciais.csv 500 0.0001 assign.csv centroids.csv
cat centroids.csv



Gerar gráfico de convergência do SSE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

sse_df = pd.read_csv('sse_evolution_seq.csv', header=None, names=['Iteration', 'SSE'])

plt.figure(figsize=(10, 6))
plt.plot(sse_df['Iteration'], sse_df['SSE'])
plt.xlabel('Iteração')
plt.ylabel('SSE (Sum of Squared Errors)')
plt.title('Convergência versão sequencial')
plt.grid(True)
plt.show()